In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import altair as alt

In [3]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [4]:
hmrc = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/UK tax/HM Revenue and Customs Receipts.csv')

In [5]:
hmrc.columns

Index(['Table 2 HMRC Receipts by financial year', 'Total Paid Over',
       'Total HMRC Receipts', 'Income Tax',
       'Pay As You Earn Income Tax (included within Income Tax)',
       'Self-Assessment Income Tax (included within Income Tax)',
       'Capital Gains Tax', 'Apprenticeship Levy',
       'National Insurance Contributions', 'PAYE NIC1 (EMP'er)',
       'PAYE NIC1 (EMP'ee)', 'SA NIC2&4', 'Value Added Tax', 'Corporation Tax',
       'Offshore (included within Corporation Tax)', 'Bank Levy',
       'Bank Surcharge', 'Diverted Profits Tax', 'Digital Services Tax',
       'Residential Property Developer Tax', 'Energy Profits Levy',
       'Electricity Generators Levy', 'Economic Crime Levy',
       'Bank payroll tax', 'Petroleum Revenue Tax',
       'Hydrocarbon Oil (Fuel duties)', 'Inheritance Tax', 'Stamp Duty Shares',
       'Stamp Duty Land Tax', 'Annual Tax on Enveloped Dwellings',
       'Tobacco Duties', 'Spirits Duties', 'Beer Duties', 'Wines Duties',
       'Cider Duti

In [6]:
sins = hmrc[['Table 2 HMRC Receipts by financial year', 'Total HMRC Receipts', 'Tobacco Duties', 'Spirits Duties', 'Beer Duties', 'Wines Duties',
       'Cider Duties', 'Betting & Gaming']]

In [7]:
sins['Total HMRC Receipts'] = sins['Total HMRC Receipts'].replace(',', '', regex=True).astype(float)
sins['Tobacco Duties'] = sins['Tobacco Duties'].replace(',', '', regex=True).astype(float)
sins['Spirits Duties'] = sins['Spirits Duties'].replace(',', '', regex=True).astype(float)
sins['Beer Duties'] = sins['Beer Duties'].replace(',', '', regex=True).astype(float)
sins['Wines Duties'] = sins['Wines Duties'].replace(',', '', regex=True).astype(float)
sins['Cider Duties'] = sins['Cider Duties'].replace(',', '', regex=True).astype(float)
sins['Betting & Gaming'] = sins['Betting & Gaming'].replace(',', '', regex=True).astype(float)

In [8]:
sins['Alcohol Duties'] = sins['Spirits Duties'] + sins['Beer Duties'] + sins['Wines Duties'] + sins['Cider Duties']

In [27]:
sins['Tobacco Duties Share'] = sins['Tobacco Duties'] / sins['Total HMRC Receipts']
sins['Alcohol Duties Share'] = sins['Alcohol Duties'] / sins['Total HMRC Receipts']
sins['Spirits Duties Share'] = sins['Spirits Duties'] / sins['Total HMRC Receipts']
sins['Beer Duties Share'] = sins['Beer Duties'] / sins['Total HMRC Receipts']
sins['Wines Duties Share'] = sins['Wines Duties'] / sins['Total HMRC Receipts']
sins['Cider Duties Share'] = sins['Cider Duties'] / sins['Total HMRC Receipts']
sins['Betting & Gaming Share'] = sins['Betting & Gaming'] / sins['Total HMRC Receipts']

In [28]:
sins_cum = sins[['Table 2 HMRC Receipts by financial year', 'Tobacco Duties', 'Spirits Duties', 'Beer Duties', 'Wines Duties',
       'Cider Duties', 'Betting & Gaming']]
sins_share = sins[['Table 2 HMRC Receipts by financial year', 'Tobacco Duties Share', 'Spirits Duties Share', 'Beer Duties Share', 'Wines Duties Share', 'Betting & Gaming Share']] 
sins_cum_s = sins[['Table 2 HMRC Receipts by financial year', 'Tobacco Duties', 'Alcohol Duties', 'Betting & Gaming']]
sins_share_s = sins[['Table 2 HMRC Receipts by financial year', 'Tobacco Duties Share', 'Alcohol Duties Share', 'Betting & Gaming Share']]

In [43]:
sins_cum_melt =sins_cum.melt(id_vars='Table 2 HMRC Receipts by financial year').rename(columns={'Table 2 HMRC Receipts by financial year':'Year', 'variable':'Tax', 'value':'Revenue (£m)'})
sins_share_melt =sins_share.melt(id_vars='Table 2 HMRC Receipts by financial year').rename(columns={'Table 2 HMRC Receipts by financial year':'Year', 'variable':'Tax', 'value':'Revenue (£m)'})
sins_cum_s_melt =sins_cum_s.melt(id_vars='Table 2 HMRC Receipts by financial year').rename(columns={'Table 2 HMRC Receipts by financial year':'Year', 'variable':'Tax', 'value':'Revenue (£m)'})
sins_share_s_melt =sins_share_s.melt(id_vars='Table 2 HMRC Receipts by financial year').rename(columns={'Table 2 HMRC Receipts by financial year':'Year', 'variable':'Tax', 'value':'Revenue (£m)'})

In [ ]:
sins_cum_s_melt['Year'] = 'FY' + sins_cum_s_melt['Year'].str.split('to 20').str[1]

In [71]:
sins_cum_s_melt['Tax'] = sins_cum_s_melt['Tax'].replace({'Tobacco Duties': 'Tobacco', 'Alcohol Duties': 'Alcohol'})

In [79]:
# 2. Find last year for each Type
last_points = sins_cum_s_melt.loc[sins_cum_s_melt.groupby('Tax')['Year'].idxmax()]

lines = alt.Chart(sins_cum_s_melt).mark_line().encode(
    x=alt.X('Year:O', axis=alt.Axis(
        labelAngle=0,
        labelFontSize=14,
        values=['FY07', 'FY09', 'FY11', 'FY13', 'FY15', 'FY17', 'FY19', 'FY21', 'FY23', 'FY25']  # Specify exact years
    )),
    y=alt.Y('Revenue (£m):Q',axis=alt.Axis(format=',.0f', labelFontSize=14, labelExpr='"£" + datum.value / 1000 + " bn"')),
    color=alt.Color('Tax:N', legend=None),
    tooltip=['Year', 'Tax', 'Revenue (£m)']
).properties(
    height=400,
    width=600,
)

# 4. Add text labels at the end
text = alt.Chart(last_points).mark_text(
    align='left',
    dx=5,
    fontWeight='bold',
    fontSize=15
).encode(
    x=alt.X('Year:N'),
    y=alt.Y('Revenue (£m):Q'),
    text='Tax',
    color=alt.Color('Tax:N', legend=None)
)

footnote = alt.Chart(pd.DataFrame({'text': [
    "Sources: HMRC tax receipts"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=13,
    fontStyle='italic',
    dy=225,
    x=-12.5
).encode(
    text='text'
)

# 5. Combine and display
chart = (lines + text + footnote).properties(title= 'Revenue Generated from "Sin" Taxes', width=500).configure_title(
    fontSize=20  # Increase title font size here
)
chart

alt.LayerChart(...)

In [80]:
# Save to png
chart.save('Sin_taxes.png', scale_factor=2)
chart.save('Sin_taxes.json', scale_factor=2)

Tax revenue from Tobacco has dropped sharply, while Alcohol & Betting continue to grow. Likely drivers: falling smoking rates + rise of yet untaxed vaping, alongside growing demand for gambling.